In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
pwd

'c:\\Users\\ten\\Desktop\\Laptop_Price\\research'

In [4]:
os.chdir("../")

In [5]:
pwd

'c:\\Users\\ten\\Desktop\\Laptop_Price'

In [6]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/kharramahendra/Laptop-Price---End-to-End-ML-Project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="kharramahendra"
os.environ["MLFLOW_TRACKING_PASSWORD"]="25511b164852982b155d2aeec465a612062ab5cf"

In [9]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    # x_train_path: Path
    # x_test_path: Path
    # y_train_path: Path
    # y_test_path: Path
    transformer_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [8]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            transformer_path=config.transformer_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/kharramahendra/Laptop-Price---End-to-End-ML-Project.mlflow",
           
        )

        return model_evaluation_config


In [11]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [12]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    


    def log_into_mlflow(self,x_test,y_test):
        
        # transformer = joblib.load(self.config.model_path)
        model = joblib.load(self.config.model_path)


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(x_test)

            (rmse, mae, r2) = self.eval_metrics(y_test, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="KNeighborsRegressormodel")
            else:
                mlflow.sklearn.log_model(model, "model")

    


In [ ]:
# I have made link between data transformation and model evaluation this code wont work

try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e